In [1]:
from KGNET import KGNET

/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kgnet=KGNET(KG_endpointUrl='http://206.12.98.118:8890/sparql',KG_NamedGraph_IRI='https://dblp2022.org',KG_Prefix='dblp2022')

## visualize graph schema

In [ ]:
types_df=kgnet.getKGNodeEdgeTypes(write_to_file=True,prefix='dblp2022')
types_df=types_df[(~types_df["object"].str.endswith("_Obj")) | (types_df["object"].str.endswith("publishedIn_Obj"))]
#types_df=types_df[~types_df["object"].str.endswith("_Obj") & types_df["object"].str.contains("publishedIn")]
g=kgnet.visualizeKG(types_df,Notebook=True,
                    width="100%",height="300px")
g.show('nx.html')

In [ ]:
types_df

## Training Pipeline

In [ ]:
taskId,modelId,trainMetadata=kgnet.train_GML(operatorType=KGNET.GML_Operator_Types.NodeClassification,
                                             GNNMethod=KGNET.GNN_Methods.Graph_SAINT,
                                             targetNodeType="dblp2022:Publication",labelNodeType="dblp2022:publishedIn_Obj")

### Query Task Info from KGMeta

In [ ]:
kgnet.KGMeta_Governer.getGMLTaskBasicInfoByID(taskId)

### Query Task Trained Models Info from KGMeta
<li> The green color indicates the best model to choose.</li>
<li> The orange color indicates the model selected by KGNET Inference query optimizer which balance between infernce time and model accuracy. </li>

In [ ]:
df,styler_df=kgnet.KGMeta_Governer.getGMLTaskModelsBasicInfoByID(taskId)
styler_df

## Inference Pipeline
write the SPARQL-ML inference query then execute it. KGNet will choose the model that match your task.

In [3]:
query = """
            prefix dblp2022:<https://dblp.org/rdf/schema#>
            prefix kgnet:<http://kgnet/>
            select ?Publication ?Title ?Org_Venue ?Pred_Venue
            from <https://dblp2022.org>
            where
            {
            ?Publication a dblp2022:Publication .
            ?Publication ?NodeClassifier ?Pred_Venue .            
            ?Publication  dblp2022:publishedIn ?Org_Venue .
            ?Publication  dblp2022:title ?Title .
            ?NodeClassifier a <kgnet:types/NodeClassifier>.
            ?NodeClassifier <kgnet:targetNode> dblp2022:Publication.
            ?NodeClassifier <kgnet:labelNode> dblp2022:publishedIn_Obj.
            }
            limit 10
        """


In [4]:
res_df,inferenceQueryMetadata=kgnet.executeSPARQLMLInferenceQuery(query)
print("The task selected model ID is:",inferenceQueryMetadata["model_id"])

The task selected model ID is: 83


In [5]:
styler_df=res_df.style.apply(kgnet.utils.highlightDiferrentRowValues, subset=['Org_Venue','Pred_Venue'], axis=1)
styler_df

,Publication,Title,Org_Venue,Pred_Venue
0,"""https://dblp.org/rec/conf/ctrsa/Bultel22""","""CCA Secure A Posteriori Openable Encryption in the Standard Model.""","""CT-RSA""",612
1,"""https://dblp.org/rec/conf/ctrsa/CaoSCCFW22""","""Lattice-Based Fault Attacks on Deterministic Signature Schemes of ECDSA and EdDSA.""","""CT-RSA""",302
2,"""https://dblp.org/rec/conf/ctrsa/Chevallier-Mames22""","""A Pairing-Free Signature Scheme from Correlation Intractable Hash Function and Strong Diffie-Hellman Assumption.""","""CT-RSA""",597
3,"""https://dblp.org/rec/conf/ctrsa/Chi-DominguezR22""","""Fully Projective Radical Isogenies in Constant-Time.""","""CT-RSA""",352
4,"""https://dblp.org/rec/conf/ctrsa/CidIR22""","""FASTA - A Stream Cipher for Fast FHE Evaluation.""","""CT-RSA""",265
5,"""https://dblp.org/rec/conf/ctrsa/CuiHWW22""","""Integral Attacks on Pyjamask-96 and Round-Reduced Pyjamask-128.""","""CT-RSA""",352
6,"""https://dblp.org/rec/conf/ctrsa/FouotsaP22""","""A New Adaptive Attack on SIDH.""","""CT-RSA""",352
7,"""https://dblp.org/rec/conf/ctrsa/Rosie22""","""Adaptively Secure Laconic Function Evaluation for NC1.""","""CT-RSA""",352
8,"""https://dblp.org/rec/conf/ctrsa/WuX22""","""More Accurate Geometric Analysis on the Impact of Successful Decryptions for IND-CCA Secure Ring/Mod-LWE/LWR Based Schemes.""","""CT-RSA""",352
9,"""https://dblp.org/rec/conf/padl/2022""","""Practical Aspects of Declarative Languages - 24th International Symposium, PADL 2022, Philadelphia, PA, USA, January 17-18, 2022, Proceedings""","""Lecture Notes in Computer Science""",655


In [ ]:
#for key in inferenceQueryMetadata:
#    print("####################"+key+"#######################")
#    print(inferenceQueryMetadata[key])